# AutoChef

## Prepare environment and process data

Data Source: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews?resource=download

Data Source v2: https://app.roboflow.com/bens-workspace-3xdyh/fridge-detection-aymme/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true

In [12]:
import os
from ultralytics import YOLO
import torch
import cv2
import requests

## Take 2 - use YOLO for a simplified approach

In [11]:

print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

print(torch.__version__)
print(torch.version.cuda)


CUDA Available: True
Device Name: NVIDIA GeForce GTX 1660
2.3.1+cu118
11.8
Torch: 2.3.1+cu118
Torchvision: 0.18.1+cu118
CUDA available: True
tensor([0], device='cuda:0')


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="8zoDBiAT84ugEQEfOSDf")
project = rf.workspace("bens-workspace-3xdyh").project("food-item-detection-fggyf-j86bp")
version = project.version(1)
dataset = version.download("yolov8")


### Core functionality

In [3]:
# Load your trained model
model = YOLO('best_v2.pt')

# Load an image
img = cv2.imread('fridge_test_2.jpg')

# Run inference
results = model(img)

# Print detected objects
for result in results:
    boxes = result.boxes
    for box in boxes:
        class_idx = int(box.cls)
        class_name = model.names[class_idx]
        confidence = float(box.conf)
        print(f"Class: {class_name}, Confidence: {confidence:.2f}, Box: {box.xyxy}")

confidence_threshold = 0.7

for result in results:
    # Filter boxes by confidence
    high_conf_boxes = [box for box in result.boxes if float(box.conf) > confidence_threshold]
    if high_conf_boxes:
        # Optionally, update result.boxes to only include high confidence boxes
        result.boxes = high_conf_boxes
        result.show()


0: 640x640 1 apple, 1 butter, 1 carrot, 1 chicken, 1 corn, 1 green_beans, 1 ground_beef, 1 ham, 1 spinach, 2 sugars, 1 tomato, 14.5ms
Speed: 8.2ms preprocess, 14.5ms inference, 322.7ms postprocess per image at shape (1, 3, 640, 640)
Class: tomato, Confidence: 0.94, Box: tensor([[317.9568, 268.1198, 425.1532, 314.7433]], device='cuda:0')
Class: apple, Confidence: 0.94, Box: tensor([[173.8641, 120.6684, 282.8015, 157.4133]], device='cuda:0')
Class: chicken, Confidence: 0.93, Box: tensor([[168.5426, 258.2592, 292.6954, 313.8521]], device='cuda:0')
Class: spinach, Confidence: 0.92, Box: tensor([[167.8266, 362.3416, 339.5656, 416.4059]], device='cuda:0')
Class: sugar, Confidence: 0.88, Box: tensor([[420.7213, 491.2953, 489.9366, 570.9257]], device='cuda:0')
Class: ground_beef, Confidence: 0.86, Box: tensor([[252.9537, 538.1412, 357.7353, 592.6963]], device='cuda:0')
Class: corn, Confidence: 0.85, Box: tensor([[472.1060, 121.4534, 525.9878, 157.7587]], device='cuda:0')
Class: green_beans, C

In [4]:
# Get a list of detected classes
detected_classes = [model.names[int(box.cls)] for box in results[0].boxes if float(box.conf) > confidence_threshold]
print("Detected classes:", detected_classes)

Detected classes: ['tomato', 'apple', 'chicken', 'spinach', 'sugar', 'ground_beef', 'corn', 'green_beans', 'butter', 'sugar', 'ham']


In [6]:
# Get recipes that can be made with the detected ingredients
url = "https://api.spoonacular.com/recipes/findByIngredients"

params = {
    "ingredients": ",".join(detected_classes),
    "ignorePantry": True,
    "ranking": 2,
    "number": 5,
    'apiKey': os.getenv("SPOONACULAR_API_KEY")
}

print("Requesting URL with params:", params)
response = requests.get(url, params=params)

response = requests.get(
    url,
    params=params
)

print(response.json())
data = response.json()

Requesting URL with params: {'ingredients': 'tomato,apple,chicken,spinach,sugar,ground_beef,corn,green_beans,butter,sugar,ham', 'ignorePantry': True, 'ranking': 2, 'number': 5, 'apiKey': '6ce254339e924106b17f13f67dcf1146'}
[{'id': 769775, 'title': 'Simple Spinach and Tomato Frittata', 'image': 'https://img.spoonacular.com/recipes/769775-312x231.jpg', 'imageType': 'jpg', 'usedIngredientCount': 2, 'missedIngredientCount': 2, 'missedIngredients': [{'id': 99037, 'amount': 1.0, 'unit': 'small', 'unitLong': 'small', 'unitShort': 'small', 'aisle': 'Pasta and Rice', 'name': 'personal skillet', 'original': 'small personal skillet', 'originalName': 'personal skillet', 'meta': [], 'image': 'https://img.spoonacular.com/ingredients_100x100/no.jpg'}, {'id': 1123, 'amount': 2.0, 'unit': '', 'unitLong': '', 'unitShort': '', 'aisle': 'Milk, Eggs, Other Dairy', 'name': 'eggs', 'original': '2-3 lightly beaten eggs', 'originalName': 'lightly beaten eggs', 'meta': ['lightly beaten'], 'image': 'https://img.

In [10]:
recommended_recipes = []
for recipes in data:
    print(f"Recipe ID: {recipes['id']}, Title: {recipes['title']}, Used Ingredients: {len(recipes['usedIngredients'])}, Missed Ingredients: {len(recipes['missedIngredients'])}")
    print("Used Ingredients:", [ingredient['name'] for ingredient in recipes['usedIngredients']])
    print("Missed Ingredients:", [ingredient['name'] for ingredient in recipes['missedIngredients']])
    print("-" * 40)
    recommended_recipes.append({
        "id": recipes['id'],
        "image": recipes['image'],
        "title": recipes['title'],
        "used_ingredients": [ingredient['name'] for ingredient in recipes['usedIngredients']],
        "missed_ingredients": [ingredient['name'] for ingredient in recipes['missedIngredients']]
    })

Recipe ID: 769775, Title: Simple Spinach and Tomato Frittata, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['cherry tomatoes', 'spinach leaves']
Missed Ingredients: ['personal skillet', 'eggs']
----------------------------------------
Recipe ID: 663748, Title: Traditional Apple Tart, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['granny smiths', 'butter']
Missed Ingredients: ['blackcurrant jam', 'oz short crust pastry - ready made if you want']
----------------------------------------
Recipe ID: 640089, Title: Corn on the Cob in Cilantro and Lime Butter, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['butter', 'corn on the cob']
Missed Ingredients: ['cilantro', 'lime']
----------------------------------------
Recipe ID: 645733, Title: Grilled Ham and Swiss Sandwich, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['pats of butter', 'ham']
Missed Ingredients: ['swiss cheese', 'bread']
----------------------------------------
Re